In [ ]:
# DataFrame
import pandas as pd

# Matplot
import matplotlib.pyplot as plt
%matplotlib inline

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
import torch

# Word2vec
import gensim

# Utility
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools

# Set log
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
russian_stopwords = stopwords.words('russian')

In [ ]:
n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
positive_data = pd.read_csv('/content/drive/MyDrive/Memento/Mood Tracker/data/positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
negative_data = pd.read_csv('/content/drive/MyDrive/Memento/Mood Tracker/data/negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
sample_size = min(positive_data.shape[0], negative_data.shape[0])

In [ ]:
raw_data = np.concatenate((positive_data['text'].values[:sample_size],
                           negative_data['text'].values[:sample_size]), axis=0)

In [ ]:
labels = [1] * sample_size + [0] * sample_size

In [ ]:
import re

def preprocess_text(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)
    text = re.sub('@[^\s]+', '', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()

In [ ]:
data = [preprocess_text(t) for t in raw_data]

In [ ]:
data[1]

'да все таки он немного похож на него но мой мальчик все равно лучше d'

In [ ]:
raw_data[1]

'Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D'

In [ ]:
import logging
import multiprocessing
import gensim
from gensim.models import Word2Vec

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
w2v = Word2Vec.load('/content/drive/MyDrive/Memento/Mood Tracker/w2v/tweets_model.w2v')

2022-04-26 21:31:35,454 : INFO : loading Word2Vec object from /content/drive/MyDrive/Memento/Mood Tracker/w2v/tweets_model.w2v
2022-04-26 21:31:42,970 : INFO : loading wv recursively from /content/drive/MyDrive/Memento/Mood Tracker/w2v/tweets_model.w2v.wv.* with mmap=None
2022-04-26 21:31:42,972 : INFO : loading vectors from /content/drive/MyDrive/Memento/Mood Tracker/w2v/tweets_model.w2v.wv.vectors.npy with mmap=None
2022-04-26 21:31:52,515 : INFO : setting ignored attribute vectors_norm to None
2022-04-26 21:31:52,522 : INFO : loading vocabulary recursively from /content/drive/MyDrive/Memento/Mood Tracker/w2v/tweets_model.w2v.vocabulary.* with mmap=None
2022-04-26 21:31:52,526 : INFO : loading trainables recursively from /content/drive/MyDrive/Memento/Mood Tracker/w2v/tweets_model.w2v.trainables.* with mmap=None
2022-04-26 21:31:52,527 : INFO : loading syn1neg from /content/drive/MyDrive/Memento/Mood Tracker/w2v/tweets_model.w2v.trainables.syn1neg.npy with mmap=None
2022-04-26 21:32:

In [ ]:
import pickle
with open('/content/drive/MyDrive/Memento/Mood Tracker/w2v/word2index.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(word2index, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
w2v.wv.add(['<PAD>', '<UNK>'], [np.zeros(200), np.zeros(200)])

In [ ]:
word2index = {token: token_index for token_index, token in enumerate(w2v.wv.index2word)}

In [ ]:
word2index['<PAD>']

1027561

In [ ]:
len(word2index)

1027563

In [ ]:
w2v_weights = torch.FloatTensor(w2v.wv.vectors)

In [ ]:
UNK_IDX = w2v_weights.shape[0] - 1
PAD_IDX = w2v_weights.shape[0] - 2

In [ ]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    def __init__(self, embedding_weights, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding.from_pretrained(embedding_weights, padding_idx=pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout=dropout, batch_first=True)
        
        # self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        # pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'), batch_first=True)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        # hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        hidden = self.dropout(hidden[0, :, :])
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

    def inference(self, text):
        embedded = self.embedding(text)
        packed_output, (hidden, cell) = self.rnn(embedded)
        # hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
        hidden = hidden[0, :, :]
        return torch.sigmoid(self.fc(hidden))

In [ ]:
embedding_dim = w2v_weights.shape[1]
hidden_dim = 256
output_dim = 1
n_layers = 1
bidirectional = False
dropout = 0.3

In [ ]:
model = SentimentRNN(w2v_weights, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, PAD_IDX)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
from copy import copy

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from tqdm import tqdm

In [ ]:
def preprocess_text(text, stopwords):
    indices = []
    tokens = word_tokenize(text, language='russian')
    for token in tokens:
        if token not in stopwords:
            word_index = w2v.wv.vocab.get(token)
            if word_index is None:
                word_index = UNK_IDX
            else:
                word_index = word_index.index
            indices.append(word_index)
    return indices

In [ ]:
def preprocess_data(data, stopwords, max_length=50):
    data = copy(data)
    lengths = []
    for i, text in enumerate(tqdm(data)):
        indices = preprocess_text(text, stopwords)
        lengths.append(len(indices))
        padded_indices = indices + [PAD_IDX] * (max_length - len(indices))
        data[i] = padded_indices
    return data, lengths

In [ ]:
processed_data, lenghts = preprocess_data(data, russian_stopwords)

100%|██████████| 223846/223846 [01:13<00:00, 3048.14it/s]


In [ ]:
from torch.utils.data import Dataset, DataLoader

class SentimentDataset(Dataset):
    def __init__(self, data, lengths, labels):
        self.data = data
        self.lengths = lengths
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.LongTensor(self.data[idx]), self.lengths[idx], self.labels[idx]

In [ ]:
len(processed_data[0])

50

In [ ]:
processed_data 

In [ ]:
processed_data[:train_end_idx]

In [ ]:
drop_idx = [i for i, l in enumerate(lenghts) if l == 0]

In [ ]:
lenghts = [l for i, l in enumerate(lenghts) if i not in drop_idx]

In [ ]:
labels = [l for i, l in enumerate(labels) if i not in drop_idx]

In [ ]:
processed_data = [l for i, l in enumerate(processed_data) if i not in drop_idx]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_data, test_data, train_lengths, test_lengths, train_labels, test_labels = train_test_split(processed_data, lenghts, labels, test_size=0.2, random_state=42)

In [ ]:
train_dataset = SentimentDataset(train_data, train_lengths, train_labels)
val_dataset = SentimentDataset(test_data, test_lengths, test_labels)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

In [ ]:
val_loader = DataLoader(val_dataset, batch_size=256)

In [ ]:
model = SentimentRNN(w2v_weights, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, PAD_IDX)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Memento/Mood Tracker/weights/model.pt'))

<All keys matched successfully>

In [ ]:
model.eval()
model.inference(torch.LongTensor([preprocess_text('ну сегодня день прошел как обычно', russian_stopwords)]))

tensor([[0.3284]], grad_fn=<SigmoidBackward0>)

In [ ]:
criterion = nn.BCEWithLogitsLoss()
device='cuda'
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def sort_sequences(inputs, lengths, labels):
    """sort_sequences
    Sort sequences according to lengths descendingly.

    :param inputs (Tensor): input sequences, size [B, T, D]
    :param lengths (Tensor): length of each sequence, size [B]
    """
    lengths_sorted, sorted_idx = lengths.sort(descending=True)
    _, unsorted_idx = sorted_idx.sort()
    return inputs[sorted_idx], lengths_sorted, labels[sorted_idx]

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in tqdm(iterator):
        
        optimizer.zero_grad()
        
        text, text_lengths, label = sort_sequences(*batch)
        text = text.to(device)
        label = label.float().to(device)
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, label)
        
        acc = binary_accuracy(predictions, label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in tqdm(iterator):

            text, text_lengths, label = sort_sequences(*batch)
            text = text.to(device)
            label = label.float().to(device)
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, label)
            
            acc = binary_accuracy(predictions, label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
def sort_sequences(inputs, lengths, labels):
    """sort_sequences
    Sort sequences according to lengths descendingly.

    :param inputs (Tensor): input sequences, size [B, T, D]
    :param lengths (Tensor): length of each sequence, size [B]
    """
    lengths_sorted, sorted_idx = lengths.sort(descending=True)
    _, unsorted_idx = sorted_idx.sort()
    return inputs[sorted_idx], lengths_sorted, labels[sorted_idx]

In [ ]:
N_EPOCHS = 100

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_loader, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/content/drive/MyDrive/Memento/Mood Tracker/weights/model_light.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

100%|██████████| 175/175 [00:01<00:00, 172.51it/s]


Epoch: 01 | Epoch Time: 0m 12s
	Train Loss: 0.527 | Train Acc: 72.61%
	 Val. Loss: 0.489 |  Val. Acc: 75.44%


100%|██████████| 175/175 [00:01<00:00, 146.09it/s]


Epoch: 02 | Epoch Time: 0m 12s
	Train Loss: 0.493 | Train Acc: 74.98%
	 Val. Loss: 0.476 |  Val. Acc: 76.18%


100%|██████████| 175/175 [00:01<00:00, 144.07it/s]


...

Epoch: 98 | Epoch Time: 0m 12s
	Train Loss: 0.313 | Train Acc: 85.95%
	 Val. Loss: 0.571 |  Val. Acc: 76.10%


100%|██████████| 175/175 [00:00<00:00, 176.01it/s]


Epoch: 99 | Epoch Time: 0m 12s
	Train Loss: 0.310 | Train Acc: 86.21%
	 Val. Loss: 0.570 |  Val. Acc: 76.20%


100%|██████████| 175/175 [00:00<00:00, 180.61it/s]

Epoch: 100 | Epoch Time: 0m 12s
	Train Loss: 0.311 | Train Acc: 86.07%
	 Val. Loss: 0.576 |  Val. Acc: 76.09%


In [ ]:
!nvidia-smi

Tue Apr 26 22:01:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    74W / 149W |   3009MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------